# Install llm lingua
####  pip install llmlingua


In [1]:
!pip install llmlingua -q

In [2]:
!pip install accelerate torch -q

In [3]:
import json
from llmlingua import PromptCompressor
import tiktoken  # For token counting
import os

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/NLU_FinalProject/Data/JSONL_Formatted/SAT_ACT/')

In [4]:
# Initialize LLMLingua-2 compressor
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    use_llmlingua2=True
)

# Initialize token counter (using OpenAI's tiktoken)
encoding = tiktoken.get_encoding("cl100k_base")

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [15]:
# Load the JSONL file
#change path and file name
input_file = "/kaggle/input/satact/SATACT_v3_tst.jsonl"  # Replace with your input file name if different
data = []
with open(input_file, "r") as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [16]:
import pandas as pd
df = pd.DataFrame(data)
len(df)

264

In [17]:
df.head(5)

,context,question,answerA,answerB,answerC,answerD,correct
0,The following text is adapted from Paul Lauren...,Which choice best states the main purpose of t...,It illustrates a character’s reaction to a new...,It explains why a character has traveled to a ...,It compares a character’s thoughts about an ev...,It presents a character feeling regret over le...,A
1,Painter Alma W. Thomas was fascinated by the c...,Which choice completes the text with the most ...,restricted,announced,distracted,influenced,D
2,"While researching a topic, a student has taken...",The student wants to emphasize the duration an...,At the Gullah Museums in Hilton Head Island an...,Louise Miller Cohen and Vermelle Rodrigues hav...,"Since 2003, Louise Miller Cohen and Vermelle R...",Influenced by the traditions of West and Centr...,C
3,The works of Chicana artist Ester Hernandez ar...,Which choice completes the text with the most ...,invented,adjusted,featured,recommended,C
4,Roasted green chiles are a popular ingredient ...,"Which finding, if true, would most directly su...",The temperature inside the roasting drum is di...,Attempts to roast green chiles using 50 helios...,Green chile connoisseurs prefer the flavor of ...,The skins of solar-roasted green chiles are ea...,B


In [18]:
# Check num of fill in the blank type of Qs : ' _', '______'
# Count rows with underscores in the "context" column
underscore_count = sum(1 for item in data if "_" in item["context"])

print(f"Number of rows with underscores in the 'context' column: {underscore_count}")

Number of rows with underscores in the 'context' column: 133


In [19]:
# Function to compress a single prompt
def compress_prompt(text):
    if "[BLANK]" in text:
        # Split the context into two parts around the token
        parts = text.split("[BLANK]")
        before_token = parts[0]
        after_token = parts[1]
        
        # Compress each part individually
        compressed_before = llm_lingua.compress_prompt(before_token)["compressed_prompt"]
        compressed_after = llm_lingua.compress_prompt(after_token)["compressed_prompt"]
        
        # Concatenate the compressed parts with the token
        compressed_context = compressed_before + "[BLANK]" + compressed_after
    else:
        # Simple compression if the token is not present
        compressed_context = llm_lingua.compress_prompt(text)["compressed_prompt"]
    
    return compressed_context


# Process each item in the JSON data
for item in data:
    #replace("_", "[BLANK]")
    item["context"] = item["context"].replace("______", "_")
    item["context"] = item["context"].replace("_", "[BLANK]")
    original_prompt = item["context"]

    # Compress the prompt
    compressed_prompt = compress_prompt(original_prompt)

    # Calculate token sizes
    original_token_size = len(encoding.encode(original_prompt))
    compressed_token_size = len(encoding.encode(compressed_prompt))

    # Add new fields to the item while preserving existing ones
    item["compressed_prompt"] = compressed_prompt
    item["token_size_before"] = original_token_size
    item["token_size_after"] = compressed_token_size

# Save the updated data to a new JSON file
output_file = "compressed_prompts_satact_v3_tst.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")

# Optional: Print summary statistics
total_original_tokens = sum(item["token_size_before"] for item in data)
total_compressed_tokens = sum(item["token_size_after"] for item in data)
print(f"\nSummary Statistics:")
print(f"Total original tokens: {total_original_tokens}")
print(f"Total compressed tokens: {total_compressed_tokens}")
print(f"Avg compression ratio: {total_original_tokens / total_compressed_tokens:.2f}x")



os.chdir('/kaggle/working')
os.listdir()


# Save the updated data to a new JSON file
output_file = "cmprsd_prmpt_satact_v3_tst.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")


# Save the output to an Excel file
excel_output_file = "cmprsd_prmpt_satact_v3_tst.xlsx"
df = pd.DataFrame(data)
df.to_excel(excel_output_file, index=False)
print(f"Output saved to Excel file: {excel_output_file}")

Processed data saved to compressed_prompts_satact_v3_tst.jsonl

Summary Statistics:
Total original tokens: 37334
Total compressed tokens: 17949
Avg compression ratio: 2.08x
Processed data saved to cmprsd_prmpt_satact_v3_tst.jsonl
Output saved to Excel file: cmprsd_prmpt_satact_v3_tst.xlsx



##### Summary Statistics:
- Total original tokens: 126094
- Total compressed tokens: 60462
- Avg compression ratio: 2.09x
- Processed data saved to cmprsd_prmpt_satact_v3_trn.jsonl
- Output saved to Excel file: cmprsd_prmpt_satact_v3_trn.xlsx

##### Summary Statistics:
- Total original tokens: 19620
- Total compressed tokens: 9448
- Avg compression ratio: 2.08x
- Processed data saved to cmprsd_prmpt_satact_v3_dev.jsonl
- Output saved to Excel file: cmprsd_prmpt_satact_v3_dev.xlsx

##### Summary Statistics:
- Total original tokens: 37334
- Total compressed tokens: 17949
- Avg compression ratio: 2.08x
- Processed data saved to cmprsd_prmpt_satact_v3_tst.jsonl
- Output saved to Excel file: cmprsd_prmpt_satact_v3_tst.xlsx

# Prompt Compression for RACE-H data

In [92]:
# Load the JSONL file
#change path and file name
input_file = "/kaggle/input/raceh-dataset/RACE-H_v1_trn.jsonl"  # Replace with your input file name if different
data = []
with open(input_file, "r") as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [93]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [94]:
import pandas as pd
df = pd.DataFrame(data)
len(df)

62445

In [95]:
df.head(5) 

,context,question,answerA,answerB,answerC,answerD,correct
0,"You may have heard the term ""the American Dream"". In 1848, James W. Marshall found gold in California and people began having golden dreams. That 19th century ""American Dream"" motivated the Gold Rush and gave California its nickname of the ""Golden State"".\nThe American Dream drove not only 1800s gold-rush prospectors but also waves of immigrants throughout that century and the next. People from Europe, and a large number of Chinese, arrived in the US in the 19th century hoping that in America they would find gold in the streets. But most, instead, worked as railroad labourers. They created the oldest Chinatown, in San Francisco, and gave the city a Chinese name ""the old gold hill"".\nIn the 20th century, some critics said that it was no longer possible to become prosperous through determination and hard work. Unfair education for students from poor families and racial discrimination almost made the American Dream a nightmare.\nThen, in the 1990s, California saw a new wave of dreamers in Silicon Valley. People poured their energy into the Internet. This new chapter of the American Dream attracted many business people and young talents from China and India to form start-ups and seek fortunes in America.\nBetter pay, a nice house, and a rising standard of living will always be attractive. However, the new American Dream is no longer just about money. It encourages Americans to consume wisely to protect the environment, improve the quality of life, and promote social justice.\nThe Governor of California, Arnold Schwarzenegger, has become the model of the new American Dream. After years of hard work, he grew from a poor young man from Austria into a movie superstar and then governor. Many people hope his story can save the American Dream and give California a brighter future.",It can be inferred that _ .,America's golden dream could never be realized,America's golden dream had bought great fortunes to Chinese immigrants,each period of time has its own taste of the American dream,determination and hard work are the best way to realize the American dream,C
1,"You may have heard the term ""the American Dream"". In 1848, James W. Marshall found gold in California and people began having golden dreams. That 19th century ""American Dream"" motivated the Gold Rush and gave California its nickname of the ""Golden State"".\nThe American Dream drove not only 1800s gold-rush prospectors but also waves of immigrants throughout that century and the next. People from Europe, and a large number of Chinese, arrived in the US in the 19th century hoping that in America they would find gold in the streets. But most, instead, worked as railroad labourers. They created the oldest Chinatown, in San Francisco, and gave the city a Chinese name ""the old gold hill"".\nIn the 20th century, some critics said that it was no longer possible to become prosperous through determination and hard work. Unfair education for students from poor families and racial discrimination almost made the American Dream a nightmare.\nThen, in the 1990s, California saw a new wave of dreamers in Silicon Valley. People poured their energy into the Internet. This new chapter of the American Dream attracted many business people and young talents from China and India to form start-ups and seek fortunes in America.\nBetter pay, a nice house, and a rising standard of living will always be attractive. However, the new American Dream is no longer just about money. It encourages Americans to consume wisely to protect the environment, improve the quality of life, and promote social justice.\nThe Governor of California, Arnold Schwarzenegger, has become the model of the new American Dream. After years of hard work, he grew from a poor young man from Austria into a movie superstar and then governor. Many people hope his story can save the American Dream and give California a brighter future.",Why did most of the early immigra

In [96]:
# Function to compress a single prompt
def compress_prompt(text):
    import re
    text = re.sub("-+", "[BLANK]", text)
    text = re.sub("_+", "[BLANK]", text)

    parts = text.split("[BLANK]")
    compressed_parts = []

    for part in parts:
        compressed_part = llm_lingua.compress_prompt(part)["compressed_prompt"]
        compressed_parts.append(compressed_part)

    compressed_context = "[BLANK]".join(compressed_parts)
    
    return compressed_context


# Process each item in the JSON data
for item in data:
    original_prompt = item["context"]

    # Compress the prompt
    compressed_prompt = compress_prompt(original_prompt)

    # Calculate token sizes
    original_token_size = len(encoding.encode(original_prompt))
    compressed_token_size = len(encoding.encode(compressed_prompt))

    # Add new fields to the item while preserving existing ones
    item["compressed_prompt"] = compressed_prompt
    item["token_size_before"] = original_token_size
    item["token_size_after"] = compressed_token_size

# Save the updated data to a new JSON file
output_file = "compressed_prompts_raceh_v1_trn.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")

# Optional: Print summary statistics
total_original_tokens = sum(item["token_size_before"] for item in data)
total_compressed_tokens = sum(item["token_size_after"] for item in data)
print(f"\nSummary Statistics:")
print(f"Total original tokens: {total_original_tokens}")
print(f"Total compressed tokens: {total_compressed_tokens}")
print(f"Avg compression ratio: {total_original_tokens / total_compressed_tokens:.2f}x")

os.chdir('/kaggle/working')
os.listdir()

# Save the updated data to a new JSON file
output_file = "cmprsd_prmpt_raceh_v1_trn.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")

# Save the output to an Excel file
excel_output_file = "cmprsd_prmpt_raceh_v1_trn.xlsx"
df = pd.DataFrame(data)
df.to_excel(excel_output_file, index=False)
print(f"Output saved to Excel file: {excel_output_file}")

Processed data saved to compressed_prompts_raceh_v1_trn.jsonl

Summary Statistics:
Total original tokens: 23647878
Total compressed tokens: 11584266
Avg compression ratio: 2.04x
Processed data saved to cmprsd_prmpt_raceh_v1_trn.jsonl
Output saved to Excel file: cmprsd_prmpt_raceh_v1_trn.xlsx


##### Summary Statistics:
- Total original tokens: 1308920
- Total compressed tokens: 639460
- Avg compression ratio: 2.05x
- Processed data saved to cmprsd_prmpt_raceh_v1_tst.jsonl
- Output saved to Excel file: cmprsd_prmpt_raceh_v1_tst.xlsx

##### Summary Statistics:
- Total original tokens: 1289102
- Total compressed tokens: 631767
- Avg compression ratio: 2.04x
- Processed data saved to cmprsd_prmpt_raceh_v1_dev.jsonl
- Output saved to Excel file: cmprsd_prmpt_raceh_v1_dev.xlsx

##### Summary Statistics:
- Total original tokens: 23647878
- Total compressed tokens: 11584266
- Avg compression ratio: 2.04x
- Processed data saved to cmprsd_prmpt_raceh_v1_trn.jsonl
- Output saved to Excel file: cmprsd_prmpt_raceh_v1_trn.xlsx

In [62]:
# Function to compress a single prompt
def compress_prompt(text):
    if "[BLANK]" in text:
        # Split the context into two parts around the token
        parts = text.split("[BLANK]")
        before_token = parts[0]
        after_token = parts[1]
        
        # Compress each part individually
        compressed_before = llm_lingua.compress_prompt(before_token)["compressed_prompt"]
        compressed_after = llm_lingua.compress_prompt(after_token)["compressed_prompt"]
        
        # Concatenate the compressed parts with the token
        compressed_context = compressed_before + "[BLANK]" + compressed_after
    else:
        # Simple compression if the token is not present
        compressed_context = llm_lingua.compress_prompt(text)["compressed_prompt"]
    
    return compressed_context


# Process each item in the JSON data
for item in data:
    # item["context"] = item["context"].replace("______", "_")
    # item["context"] = item["context"].replace("_", "[BLANK]")
    original_prompt = item["context"]

    # Compress the prompt
    compressed_prompt = compress_prompt(original_prompt)

    # Calculate token sizes
    original_token_size = len(encoding.encode(original_prompt))
    compressed_token_size = len(encoding.encode(compressed_prompt))

    # Add new fields to the item while preserving existing ones
    item["compressed_prompt"] = compressed_prompt
    item["token_size_before"] = original_token_size
    item["token_size_after"] = compressed_token_size

# Save the updated data to a new JSON file
output_file = "compressed_prompts_raceh_v1_tst.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")

# Optional: Print summary statistics
total_original_tokens = sum(item["token_size_before"] for item in data)
total_compressed_tokens = sum(item["token_size_after"] for item in data)
print(f"\nSummary Statistics:")
print(f"Total original tokens: {total_original_tokens}")
print(f"Total compressed tokens: {total_compressed_tokens}")
print(f"Avg compression ratio: {total_original_tokens / total_compressed_tokens:.2f}x")



os.chdir('/kaggle/working')
os.listdir()


# Save the updated data to a new JSON file
output_file = "cmprsd_prmpt_raceh_v1_tst.jsonl"
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Processed data saved to {output_file}")


# Save the output to an Excel file
excel_output_file = "cmprsd_prmpt_raceh_v1_tst.xlsx"
df = pd.DataFrame(data)
df.to_excel(excel_output_file, index=False)
print(f"Output saved to Excel file: {excel_output_file}")

Processed data saved to compressed_prompts_raceh_v1_tst.jsonl

Summary Statistics:
Total original tokens: 1311587
Total compressed tokens: 603173
Avg compression ratio: 2.17x
Processed data saved to cmprsd_prmpt_raceh_v1_tst.jsonl
Output saved to Excel file: cmprsd_prmpt_raceh_v1_tst.xlsx


In [75]:
df1 = pd.read_excel('cmprsd_prmpt_raceh_v1_tst.xlsx')
#df1.head(5)

##### Summary Statistics:
- Total original tokens: 1311587
- Total compressed tokens: 603173
- Avg compression ratio: 2.17x
- Processed data saved to cmprsd_prmpt_raceh_v1_tst.jsonl
- Output saved to Excel file: cmprsd_prmpt_raceh_v1_tst.xlsx

# Repeat the steps for all: train, validation and test sets of both SAT/ACT and RACE-H dataset